# Qubit Spectroscopy

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see [Getting Started tutorial](../../../../tutorials/sources/getting_started.ipynb) if you need to create your setup and qubits for the first time. 

You can run this notebook on real hardware in the lab. However, if you don't have the hardware at your disposal, you can also run the notebook "as is" using an emulated session (see below). 

If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background
In this how-to guide, you'll perform a measurement to determine the qubit transition frequency. The LabOne Q Applications Library provides different experiment workflows to achieve this task. You can either just sweep the frequency of a qubit drive pulse using the `qubit_spectroscopy` experiment workflow, or perform a 2D sweep to optimize at the same time the amplitude of the qubit drive pulse (the `qubit_spectroscopy_amplitude` experiment workflow).

Below is a simple diagram of showing the ground and the first excited state of a qubit, where the transition frequency corresponds to the $\omega_{01}$.

![](../../../images/qubit_ge.svg "Sketch of ground and first excited states of a qubit")

As the swept qubit drive frequency comes close to the qubit transition frequency, the qubit will be driven out of its ground state. Due to dispersive coupling (see dispersive readout how-to for more detail), the signal from the readout resonator will then experience a shift in magnitude and phase.

Plotting the readout signal, we can obtain a lorentzian lineshape as a function of the qubit drive frequency. This lorentzian will be centered around the qubit transition frequency, and from its linewidth we can approximate the qubit dephasing time. Below is an example dataplot of a qubit spectroscopy measurement:

![](../../../images/qubit_spec_result.png "Example data plot of qubit spectroscopy")

### Imports

You'll start by importing the qubit-spectroscopy experiment workflows from `laboneq_applications`, as well as `laboneq.simple` and a demo QPU and device setup to run in emulation mode.

In [ ]:
import numpy as np
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import (
    qubit_spectroscopy,
    qubit_spectroscopy_amplitude,
)
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup as shown in the [Getting Started tutorial](../../../../tutorials/sources/getting_started.ipynb).

In [ ]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Create a `FolderStore` for Saving Data

The experiment `Workflows` can automatically save the inputs and outputs of all their tasks to the folder path we specify when instantiating the `FolderStore`. Here, we choose the current working directory.

In [ ]:
# import FolderStore from the `workflow` namespace of LabOne Q, which was imported
# from `laboneq.simple`
from pathlib import Path

folder_store = workflow.logbook.FolderStore(Path.cwd())

We disable saving in this guide. To enable it, simply run `folder_store.activate()`.

In [ ]:
folder_store.deactivate()

### Running the Experiment Workflow

You'll now make the experiment workflow and run. For more details on what experiment workflows are and what tasks they execute, see the [Experiment Workflows tutorial](../../../../tutorials/sources/experiment_workflows.ipynb).

Let's first create the options class for the qubit-spectroscopy experiment and inspect it using the tree view of the option fields per task:

In [ ]:
options = qubit_spectroscopy.experiment_workflow.options()
options

Or, alternatively, using the `show_fields` function from the `workflow` namespace of LabOne Q, which was imported from `laboneq.simple`:

In [ ]:
workflow.show_fields(options)

Notice that, unless we change it:

- the experiment is run in `AcquisitionType.INTEGRATION` and `AveragingMode.CYCLIC`, using 1024 averages (`count`)
- there is a waiting time of 1$\mu$s between the end of every acquisition and the playback of the subsequent readout pulse (`spectroscopy_reset_delay`)
- the analysis workflow will run automatically (`do_analysis=True`)
- the figures produced by the analysis are automatically closed (`close_figures=False`)
- the qubit parameters will not be updated (`update=False`)

Here, let's disable closing the figures:

In [ ]:
options.close_figures(False)

Now we run the experiment workflow on the first two qubits in parallel. 

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

exp_workflow = qubit_spectroscopy.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    frequencies=[np.linspace(5.8e9, 6.2e9, 101), np.linspace(5.9e9, 6.3e9, 101)],
    options=options
)

workflow_results = exp_workflow.run()

#### Inspect the Tasks That Were Run

In [ ]:
for t in workflow_results.tasks:
    print(t)

#### Inspect the Output Simulation

You can also inspect the compiled experiment and plot the simulated output:

In [ ]:
compiled_experiment = workflow_results.tasks["compile_experiment"].output
plot_simulation(
    compiled_experiment, length=50e-6, signal_names_to_show=["measure"]
)

#### Inspecting the Source Code of the Pulse-Sequence Creation Task

You can inspect the source code of the `create_experiment` task defined in `ramsey` to see how the experiment pulse sequence is created using quantum operations. To learn more about the latter, see the Quantum Operations tutorial (add link).

In [ ]:
qubit_spectroscopy.create_experiment.src

To learn more about how to work with experiment `Workflows`, check out the [Experiment Workflows tutorial](../../../../tutorials/sources/experiment_workflows.ipynb).

Here, let's briefly inspect the analysis-workflow results.

#### Analysis Results

Let's check what tasks were run as part of the analysis workflow:

In [ ]:
analysis_workflow_results = workflow_results.tasks["analysis_workflow"]
for t in analysis_workflow_results.tasks:
    print(t)

We can access the qubit parameters extracted by the analysis from the output of the analysis-workflow:

In [ ]:
from pprint import pprint

pprint(analysis_workflow_results.output)  # noqa: T203

Check out the [Experiment Workflows tutorial](../../../../tutorials/sources/experiment_workflows.ipynb) to see how you can manually update the qubit parameters to these new values, or reset them to the old ones. 

### Sweep the amplitude at the same time

Getting back to our found parameters, we now repeat the workflow while sweeping the amplitude of the drive frequency at the same time. Since the analysis and the interface are different, a dedicated workflow is used, but the input only differs slightly. 

Since the amplitude is swept in the outer loop, while frequency is swpet in the inner loop, we set the frequency sweep to only 2 to have the amplitude modification visible in the OutputSimulator.

In [ ]:
exp_workflow_amp = qubit_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    amplitudes = [np.linspace(0.1, 0.5, 2), np.linspace(0.1, 0.5, 2)],
    frequencies=[np.linspace(5.8e9, 6.2e9, 3), np.linspace(5.9e9, 6.3e9, 3)],
)

workflow_results_amp = exp_workflow_amp.run()
compiled_experiment_amp = workflow_results_amp.tasks["compile_experiment"].output
plot_simulation(compiled_experiment_amp, length=50e-6)

Great! You've now run your qubit-spectroscopy experiment. Check out other experiments in this manual to keep characterizing your qubits.